In [13]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt

from scipy.spatial import distance as dist
from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img

In [14]:
FACE = list(range(17, 68))
FACE_COMPLETA = list(range(0, 68))
LABIO = list(range(48, 61))
SOBRANCELHA_DIREITA = list(range(17, 22))
SOBRANCELHA_ESQUERDA = list(range(22, 27))
OLHO_DIREITO = list(range(36, 42))
OLHO_ESQUERDO = list(range(42, 48))
NARIZ = list(range(27, 35))
MANDIBULA = list(range(0, 17))

In [15]:
classificador_dlib_68_path = "classificadores/shape_predictor_68_face_landmarks.dat"
classificador_dlib = dlib.shape_predictor(classificador_dlib_68_path)
detector_face = dlib.get_frontal_face_detector()

In [16]:
def anotar_rosto(imagem):
    retangulos = detector_face(imagem, 1)
    if len(retangulos) == 0:
        return None
        
    for k, d in enumerate(retangulos):
        print("identificado rosto " + str(k))
        cv2.rectangle(imagem, (d.left(), d.top()),
                      (d.right(), d.bottom()), (255, 255, 0), 2)

    return imagem

In [17]:
def pontos_marcos_faciais(imagem):
    retangulos = detector_face(imagem, 1)

    if len(retangulos) == 0:
        return None
        
    marcos = []
    for ret in retangulos:
        marcos.append(np.matrix([[p.x, p.y]
                                 for p in classificador_dlib(imagem, ret).parts()]))
    return marcos

In [18]:
def anotar_marcos_faciais(imagem, marcos):
    for marco in marcos:
        for idx, ponto in enumerate(marco):
            centro = (ponto[0, 0], ponto[0, 1])
            cv2.circle(imagem, centro, 3, (255, 255, 0), -1)
            cv2.putText(imagem, str(idx), centro,
                        cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    return imagem

In [19]:
def padronizar_imagem(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (500, 400))
    
    return frame


In [20]:
def exbir_video(frame):
    img = Img.fromarray(frame, "RGB")
    buffer = BytesIO()
    img.save(buffer, format="JPEG")
    
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)


In [21]:
def anotar_marcos_casca_convexa(imagem, marcos):
    retangulos = detector_face(imagem, 1)

    if len(retangulos) == 0:
        return None

    for idx, ret in enumerate(retangulos):
        marco = marcos[idx]

        ponto = cv2.convexHull(marco[OLHO_ESQUERDO])
        cv2.drawContours(imagem, [ponto], 0, (255, 255, 0), 2)

        ponto = cv2.convexHull(marco[OLHO_DIREITO])
        cv2.drawContours(imagem, [ponto], 0, (255, 255, 0), 2)
        
    return imagem

In [26]:
def anotar_marcos_casca_convexa_boca(imagem, marcos):
    retangulos = detector_face(imagem, 1)

    if len(retangulos) == 0:
        return None

    for idx, ret in enumerate(retangulos):
        marco = marcos[idx]

        ponto = cv2.convexHull(marco[LABIO])
        cv2.drawContours(imagem, [ponto], 0, (0, 255, 0), 2)
    return imagem

In [24]:
def aspecto_razao_olhos(pontos_olhos):
    a = dist.euclidean(pontos_olhos[1], pontos_olhos[5])
    b = dist.euclidean(pontos_olhos[2], pontos_olhos[4])
    c = dist.euclidean(pontos_olhos[0], pontos_olhos[3])
    aspecto_razao = (a + b) / (2.0 * c)

    return aspecto_razao


In [23]:
def aspecto_razao_boca(pontos_boca):
    a = dist.euclidean(pontos_boca[3], pontos_boca[9])
    b = dist.euclidean(pontos_boca[2], pontos_boca[10])
    c = dist.euclidean(pontos_boca[4], pontos_boca[8])
    d = dist.euclidean(pontos_boca[0], pontos_boca[6])  # transversal
    aspecto_razao = (a + b + c) / (3.0 * d)

    return aspecto_razao


In [22]:
video = cv2.VideoCapture("../databases/videos/expressoes.mov")
try:
    while (True):
        capture_ok, frame = video.read()
        if capture_ok:
            frame = padronizar_imagem(frame)
            exbir_video(frame)
except KeyboardInterrupt:
    video.release()
    print("Interrompido")


Interrompido


In [28]:
try:
    ar_max = 0
    video = cv2.VideoCapture("../databases/videos/bocejo.mov")

    while (True):
        capture_ok, frame = video.read()
        marcos_faciais = pontos_marcos_faciais(frame)

        if marcos_faciais is not None:
            ar_boca = aspecto_razao_boca(marcos_faciais[0][LABIO])
            ar_boca = round(ar_boca, 3)

            if ar_boca > ar_max:
                ar_max = ar_boca

            info = "boca " + str(ar_boca) + " maximo " + str(ar_max)
            frame = anotar_marcos_casca_convexa_boca(frame, marcos_faciais)

            cv2.putText(frame, info, (20, 50),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 0), 2)
        exbir_video(frame)

except KeyboardInterrupt:
    video.release()
    print("Interrompido")


Interrompido


In [30]:
try:
    min_olho_esq = 1
    min_olho_dir = 1
    video = cv2.VideoCapture("../databases/videos/olhos-fechados.mov")

    while (True):
        capture_ok, frame = video.read()
        frame = padronizar_imagem(frame)
        marcos_faciais = pontos_marcos_faciais(frame)

        if marcos_faciais is not None:
            ar_olho_esq = aspecto_razao_olhos(
                marcos_faciais[0][OLHO_ESQUERDO])
            ar_olho_dir = aspecto_razao_olhos(
                marcos_faciais[0][OLHO_DIREITO])

            ar_olho_esq = round(ar_olho_esq, 3)
            ar_olho_dir = round(ar_olho_dir, 3)

            if ar_olho_esq < min_olho_esq:
                min_olho_esq = ar_olho_esq

            if ar_olho_dir < min_olho_dir:
                min_olho_dir = ar_olho_dir

            info_esq = "Olho esquerdo " + \
                str(ar_olho_esq) + "mínimo " + str(min_olho_esq)
            info_dir = "Olho direito " + \
                str(ar_olho_dir) + "mínimo " + str(min_olho_dir)

            frame = anotar_marcos_casca_convexa(frame, marcos_faciais)
            
            cv2.putText(frame, info_esq, (20, 30),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 0), 2)

            cv2.putText(frame, info_dir, (20, 50),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 0), 2)
        exbir_video(frame)
except KeyboardInterrupt:
    video.release()
    print("Interrompido")


Interrompido
